In [ ]:
from keras.models import model_from_json
import re
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import numpy as np
import ast

def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

def embed_input(story, query, story_maxlen, query_maxlen):
    story = [tokenize(sentence) for sentence in story]
    query = tokenize(query)
    
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    story = flatten(story)
    
    vocab = sorted(set(story + query))
    #print(vocab)
    
    # Reserve 0 for masking via pad_sequences
    vocab_size = len(vocab) + 1
    
    word_dict = dict((c, i + 1) for i, c in enumerate(vocab))
    X = [word_dict[w] for w in story]
    Xq = [word_dict[w] for w in query]
    
    X = pad_sequences([X], maxlen=story_maxlen)
    Xq = pad_sequences([Xq], maxlen=query_maxlen)
    
    return X, Xq

# load json and create model
loaded_model_json = ""
with open('model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)

# load weights into model
loaded_model.load_weights("model_final.h5")
print("Loaded model from disk")

loaded_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

with open("train_config", "r") as f:
    lines = f.read().splitlines()
    story_maxlen = int(lines[0].split(": ")[1])
    query_maxlen = int(lines[1].split(": ")[1])
    word_idx = ast.literal_eval(lines[2])

# use loaded model to respond to user input from command line
user_story = input("Tell a story (period-separated sentences): ")
user_story = user_story.strip().split(". ")
user_query = []
while (user_query != 'q'):
    #print("Ask a question:")
    user_query = input("Ask a question: ")
    
    story_test, query_test = embed_input(user_story, user_query, story_maxlen, query_maxlen)
    #print('story_test:', story_test)
    #print('query_test:', query_test)
    
    resp = loaded_model.predict([story_test, query_test])
    resp = abs(np.ceil(resp-0.5))
    i, decode_index = np.where(resp==1.)
    #print(decode_index)
    
    for answer, encode in word_idx.items():
        if encode == decode_index:
            print("Answer: ", answer)

Loaded model from disk
Tell a story (period-separated sentences): Daniel picked up the apple there. Daniel went to the hallway. John moved to the office. Mary picked up the football there. Mary put down the football there. Mary picked up the football there. Daniel journeyed to the kitchen. Daniel travelled to the hallway. Daniel put down the apple. Mary moved to the kitchen.
Ask a question: where is John?


/home/longw/.virtualenvs/mie1513-py3.5/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Answer:  office


Daniel picked up the apple there. Daniel went to the hallway. John moved to the office. Mary picked up the football there. Mary put down the football there. Mary picked up the football there. Daniel journeyed to the kitchen. Daniel travelled to the hallway. Daniel put down the apple. Mary moved to the kitchen.
